<a href="https://colab.research.google.com/github/plaban1981/Dphi_hackathon/blob/main/auto_sklearn_AMIR19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto-Sklearn

auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator.
It frees a machine learning user from algorithm selection and hyperparameter tuning. It leverages recent advantages in Bayesian optimization, meta-learning and ensemble construction.

[Docu](https://automl.github.io/auto-sklearn/master/),
[Paper](https://papers.nips.cc/paper/5872-efficient-and-robust-automated-machine-learning)

Notebook author: Marius Lindauer ([www.automl.org](https://www.automl.org))

# Installation

The notebook was created based on 
scikit-learn 0.19.2, smac 0.8.0 and auto-sklearn 0.5.1.

In [ ]:
!apt-get install swig -y
!pip install Cython numpy

# sometimes you have to run the next command twice on colab
# I haven't figured out why
!pip install auto-sklearn

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
# ignore some annoying warnings for demonstrating auto-sklearn 
# shouldn't be done in real production
import numpy as np
np.warnings.filterwarnings('ignore')

# 1st Example

We want to train a classifier for the [digits](http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) dataset.
All we do is to split the dataset into training and test data,
pass the training data to auto-sklearn
and evaluate the trained model on test data.


In [ ]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

# Load data
X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)

In [ ]:
import autosklearn.classification

# configure auto-sklearn
automl = autosklearn.classification.AutoSklearnClassifier(
          time_left_for_this_task=120, # run auto-sklearn for at most 2min
          per_run_time_limit=30, # spend at most 30 sec for each model training
          )

# train model(s)
automl.fit(X_train, y_train)

# evaluate
y_hat = automl.predict(X_test)
test_acc = sklearn.metrics.accuracy_score(y_test, y_hat)
print("Test Accuracy score {0}".format(test_acc))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


[WARNING] [2019-05-14 15:52:03,539:EnsembleBuilder(1):d74860caaa557f473ce23908ff7ba369] No models better than random - using Dummy Score!
[WARNING] [2019-05-14 15:52:03,571:EnsembleBuilder(1):d74860caaa557f473ce23908ff7ba369] No models better than random - using Dummy Score!
1
['/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000000.ensemble', '/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000001.ensemble', '/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000002.ensemble', '/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000003.ensemble', '/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000004.ensemble', '/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000005.ensemble', '/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000006.ensemble', '/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000007.ensemble', '/tmp/autosklearn_tmp_707_2898/.auto-sklearn/ensembles/1.0000000008.ensemble', '/tmp/auto

The accuracy might not be quite state-of-the-art after running auto-sklearn only for two minutes. If you want to achieve better results, please try to increase the time limit `time_left_for_this_task`.

## Inspect some statistics of our first example

Please note that auto-sklearn has internally used a holdout set of the traning set to estimate the quality of the trained model. Based on this hold-out validation set, auto-sklearn reports a validation score.

In [ ]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: d74860caaa557f473ce23908ff7ba369
  Metric: accuracy
  Best validation score: 0.991011
  Number of target algorithm runs: 21
  Number of successful target algorithm runs: 18
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0



## Inspect Ensemble

Auto-sklearn considers all trained models as potential candidates to build an ensemble out of these.
The following command allows you to see the ensemble.

Since the ensemble does not use a simple majority voting, but a weighted ensemble,
the fomat is the following:

`(ensemble weight, machine learning pipeline)`

In [ ]:
print(automl.show_models())

[(0.120000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'categorical_encoding:__choice__': 'no_encoding', 'classifier:__choice__': 'lda', 'imputation:strategy': 'most_frequent', 'preprocessor:__choice__': 'polynomial', 'rescaling:__choice__': 'minmax', 'classifier:lda:n_components': 170, 'classifier:lda:shrinkage': 'None', 'classifier:lda:tol': 0.014191958374153584, 'preprocessor:polynomial:degree': 2, 'preprocessor:polynomial:include_bias': 'False', 'preprocessor:polynomial:interaction_only': 'True'},
dataset_properties={
  'task': 2,
  'sparse': False,
  'multilabel': False,
  'multiclass': True,
  'target_type': 'classification',
  'signed': False})),
(0.100000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'libsvm_svc', 'imputation:strategy': 'most_frequent', 'preprocessor:__choice__': 'no_preprocessing', 'rescaling:__choice__': 'normalize', 'categorical_encoding:one_h

# 2nd Example: Holdout resampling

Now, let's switch to a different dataset, breast_cancer

In [ ]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

import autosklearn.classification

X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

automl = autosklearn.classification.AutoSklearnClassifier(
          time_left_for_this_task=120,
          per_run_time_limit=30,
          disable_evaluator_output=False,
          resampling_strategy='holdout',
          resampling_strategy_arguments={'train_size': 0.80}
          )

automl.fit(X_train, y_train, dataset_name='breast_cancer')

y_hat = automl.predict(X_test)
test_acc = sklearn.metrics.accuracy_score(y_test, y_hat)
print("Test Accuracy score: {0}".format(test_acc))

[WARNING] [2019-05-14 15:53:59,451:EnsembleBuilder(1):breast_cancer] No models better than random - using Dummy Score!
[WARNING] [2019-05-14 15:53:59,463:EnsembleBuilder(1):breast_cancer] No models better than random - using Dummy Score!
[WARNING] [2019-05-14 15:55:39,649:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-14 15:55:39,649:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
1
['/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000000.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000001.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000002.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000003.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000004.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.000